In [1]:
from auto_login import get_curr_path,get_credentials,auto_login,generate_access_token
from macd import compute_ema
import os 
import logging 
from kiteconnect import KiteConnect
import datetime as dt
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
import ta

In [2]:
folder_name = 'api_keys'
curr_dir = get_curr_path(folder_name)

file_name = 'credentials.txt'
api_key,api_secret,user_name,pwd,totp_key = get_credentials(curr_dir,file_name)

request_token = auto_login(api_key,user_name,pwd,totp_key)

access_token = generate_access_token(request_token,api_key,api_secret)

In [3]:
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)

In [4]:
instrument_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv('NSE_instruments.csv',index=False)

In [5]:
instrument_df.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,256265,1001,NIFTY 50,NIFTY 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE
1,256777,1003,NIFTY MIDCAP 100,NIFTY MIDCAP 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
2,260105,1016,NIFTY BANK,NIFTY BANK,0.0,,0.0,0.0,0,EQ,INDICES,NSE
3,260617,1018,NIFTY 100,NIFTY 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
4,257033,1004,NIFTY DIV OPPS 50,NIFTY DIV OPPS 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE


In [6]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

In [7]:
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

In [8]:
infy_df = fetchOHLC('INFY','5minute',5)
infy_df.to_csv('infy.csv')

In [9]:
infy_df.head()

,open,high,low,close,volume
date,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865


In [93]:
df = infy_df.copy()    

In [94]:
df.head()

,open,high,low,close,volume
date,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865


In [85]:
# df['close_diff'] = df['close'].diff().dropna()

In [86]:
# df.head()

,open,high,low,close,volume,close_diff
date,,,,,,
2024-06-03 09:15:00+05:30,1440.00,1440.00,1422.50,1426.05,608537,NaN
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,-4.60
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,-2.60
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,-4.35
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,-3.70


In [87]:
# df.dropna(inplace=True)

In [88]:
# df

,open,high,low,close,volume,close_diff
date,,,,,,
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,-4.60
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,-2.60
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,-4.35
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,-3.70
2024-06-03 09:40:00+05:30,1410.65,1413.40,1410.50,1411.55,109628,0.75
...,...,...,...,...,...,...
2024-06-06 11:05:00+05:30,1450.40,1452.00,1449.05,1450.30,100546,-0.10
2024-06-06 11:10:00+05:30,1450.30,1451.50,1450.00,1450.95,41474,0.65
2024-06-06 11:15:00+05:30,1450.65,1455.00,1450.35,1454.45,179522,3.50


In [89]:
# df['gain'] = df['close_diff'].apply(lambda x: x if x > 0 else 0)
# df['loss'] = df['close_diff'].apply(lambda x: -x if x < 0 else 0)
# df

,open,high,low,close,volume,close_diff,gain,loss
date,,,,,,,,
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,-4.60,0.00,4.60
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,-2.60,0.00,2.60
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,-4.35,0.00,4.35
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,-3.70,0.00,3.70
2024-06-03 09:40:00+05:30,1410.65,1413.40,1410.50,1411.55,109628,0.75,0.75,0.00
...,...,...,...,...,...,...,...,...
2024-06-06 11:05:00+05:30,1450.40,1452.00,1449.05,1450.30,100546,-0.10,0.00,0.10
2024-06-06 11:10:00+05:30,1450.30,1451.50,1450.00,1450.95,41474,0.65,0.65,0.00
2024-06-06 11:15:00+05:30,1450.65,1455.00,1450.35,1454.45,179522,3.50,3.50,0.00


In [90]:
# df.to_csv('gain_loss.csv')

In [70]:
# initial_avg_gain = df['gain'].rolling(window=14,min_periods=14).mean()

date
2024-06-03 09:20:00+05:30         NaN
2024-06-03 09:25:00+05:30         NaN
2024-06-03 09:30:00+05:30         NaN
2024-06-03 09:35:00+05:30         NaN
2024-06-03 09:40:00+05:30         NaN
                               ...   
2024-06-06 11:05:00+05:30    1.025000
2024-06-06 11:10:00+05:30    1.071429
2024-06-06 11:15:00+05:30    1.321429
2024-06-06 11:20:00+05:30    1.328571
2024-06-06 11:25:00+05:30    1.242857
Name: gain, Length: 251, dtype: float64


In [76]:
# initial_avg_gain[0:14]

date
2024-06-03 09:20:00+05:30         NaN
2024-06-03 09:25:00+05:30         NaN
2024-06-03 09:30:00+05:30         NaN
2024-06-03 09:35:00+05:30         NaN
2024-06-03 09:40:00+05:30         NaN
2024-06-03 09:45:00+05:30         NaN
2024-06-03 09:50:00+05:30         NaN
2024-06-03 09:55:00+05:30         NaN
2024-06-03 10:00:00+05:30         NaN
2024-06-03 10:05:00+05:30         NaN
2024-06-03 10:10:00+05:30         NaN
2024-06-03 10:15:00+05:30         NaN
2024-06-03 10:20:00+05:30         NaN
2024-06-03 10:25:00+05:30    0.435714
Name: gain, dtype: float64

In [95]:
#Computing the avg_gain and avg_loss (use relative moving average)

def compute_rsi(df,n):
    df['close_diff'] = df['close'].diff().dropna()
    df.dropna(inplace=True)
    df['gain'] = df['close_diff'].apply(lambda x: x if x > 0 else 0)
    df['loss'] = df['close_diff'].apply(lambda x: -x if x < 0 else 0)
    alpha = 1/n
    
    for i in range(n,len(df)):
        if i == n:
            df.loc[df.index[i],'avg_gain'] = df['gain'].rolling(window=n,min_periods=n).mean().iloc[n-1]
            df.loc[df.index[i],'avg_loss'] = df['loss'].rolling(window=n,min_periods=n).mean().iloc[n-1]
        else:
            df.loc[df.index[i],'avg_gain'] = (alpha * df.loc[df.index[i],'gain']) + (1 - alpha)*(df.loc[df.index[i-1],'avg_gain'])
            df.loc[df.index[i],'avg_loss'] = (alpha * df.loc[df.index[i],'loss']) + (1 - alpha)*(df.loc[df.index[i-1],'avg_loss'])
    
    df['rs'] = df['avg_gain'] / df['avg_loss']
    df['rsi'] = 100 - (100/(1 + df['rs']))
    return df
            


df = compute_rsi(df,14)

        

In [96]:
df

,open,high,low,close,volume,close_diff,gain,loss,avg_gain,avg_loss,rs,rsi
date,,,,,,,,,,,,
2024-06-03 09:20:00+05:30,1425.35,1426.70,1418.45,1421.45,334479,-4.60,0.00,4.60,NaN,NaN,NaN,NaN
2024-06-03 09:25:00+05:30,1421.85,1422.25,1415.75,1418.85,150139,-2.60,0.00,2.60,NaN,NaN,NaN,NaN
2024-06-03 09:30:00+05:30,1419.00,1419.15,1413.05,1414.50,153774,-4.35,0.00,4.35,NaN,NaN,NaN,NaN
2024-06-03 09:35:00+05:30,1414.55,1415.05,1410.60,1410.80,197865,-3.70,0.00,3.70,NaN,NaN,NaN,NaN
2024-06-03 09:40:00+05:30,1410.65,1413.40,1410.50,1411.55,109628,0.75,0.75,0.00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-06 11:05:00+05:30,1450.40,1452.00,1449.05,1450.30,100546,-0.10,0.00,0.10,1.138944,0.598238,1.903830,65.562727
2024-06-06 11:10:00+05:30,1450.30,1451.50,1450.00,1450.95,41474,0.65,0.65,0.00,1.104019,0.555507,1.987409,66.526179
2024-06-06 11:15:00+05:30,1450.65,1455.00,1450.35,1454.45,179522,3.50,3.50,0.00,1.275160,0.515828,2.472067,71.198714


In [97]:
df.to_csv('rsi.csv')